# Bibliotek

In [1]:
import os
import collections
import re
import time
from tqdm import tqdm # for displaying progress bar
from multiprocess import Pool
from utils import file_read, word_search
import itertools
import pandas as pd

# Indhent søgeord fra excel-fil

In [2]:
# indhent søgeord fra excel-fil
data = pd.read_excel("Søgeord.xlsx")
searchwordslist = list(data.Søgeord)
searchwordslist

['economy',
 'covid',
 'sustainability',
 'ukraine',
 'france',
 'ukrainian',
 'sustainability',
 'emission']

## Tekst-søgning (parallel)

In [3]:
# folder
folder = "Bæredygtighedsrapporter/"

In [4]:
# tekstdokumenter
textlist = os.listdir(folder)
textlist

['A.P. Møller - Mærsk sustain.txt',
 'ALK-Abello sustain.txt',
 'Alm. brand sustain.txt',
 'Ambu sustain.txt',
 'Bavarian Nordic sustain.txt',
 'Bestseller sustain.txt',
 'Boozt sustain.txt',
 'Carlsberg sustain.txt',
 'ChemoMetec sustain.txt',
 'Chr. Hansen Holding sustain.txt',
 'Coloplast sustain.txt',
 'Danske bank sustain.txt',
 'Demant sustain.txt',
 'DFDS sustain.txt',
 'DSV sustain.txt',
 'FLSmidth sustain.txt',
 'Genmab sustain.txt',
 'GN Store Nord sustain.txt',
 'H. Lundbeck sustain.txt',
 'ISS sustain.txt',
 'Jeudan sustain.txt',
 'Jyske Bank sustain.txt',
 'Netcompany sustain.txt',
 'NKT sustain.txt',
 'Nordea Bank sustain.txt',
 'Novozymes sustain.txt',
 'Pandora sustain.txt',
 'Q8 sustain.txt',
 'Ringkjøbing Landbobank sustain.txt',
 'Rockwool sustain.txt',
 'Salling Group sustain.txt',
 'Scandinavian Tobacco Group sustain.txt',
 'Schouw & Co sustain.txt',
 'Simcorp sustain.txt',
 'Spar Nord sustain.txt',
 'Sydbank sustain.txt',
 'The Drilling Company of 1972 sustain.txt

In [5]:
tic = time.time()

# konvertér søgeordslsite + foldername til lister som er lige så lange som mappens indhold
searchwordslist = [searchwordslist]*len(textlist)
folderlist = [folder]*len(textlist)

func_arguments = list(zip(folderlist, textlist,searchwordslist))

max_pool = 4  # sæt antallet a processors 
with Pool(max_pool) as p:
    pool_outputs = list(
        tqdm(
            p.starmap(file_read, func_arguments),
            total=len(textlist)
        )
    )   
    
output_dict = dict(pool_outputs)
toc = time.time()
paralleltime = toc-tic
print(f"Time ellapsed: {paralleltime:.2f} seconds")


100%|███████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 42592.06it/s]

Time ellapsed: 1.07 seconds


In [6]:
df = pd.DataFrame(output_dict).T
df

,economy,covid,sustainability,ukraine,france,ukrainian,emission
A.P. Møller - Mærsk sustain.txt,4,43,174,0,0,0,7
ALK-Abello sustain.txt,0,9,102,0,12,0,2
Alm. brand sustain.txt,0,10,0,0,0,0,0
Ambu sustain.txt,6,6,61,0,0,0,15
Bavarian Nordic sustain.txt,0,2,4,0,0,0,9
Bestseller sustain.txt,6,28,104,0,2,0,0
Boozt sustain.txt,0,3,96,0,0,0,8
Carlsberg sustain.txt,0,54,212,11,9,0,7
ChemoMetec sustain.txt,0,55,0,0,0,0,0
Chr. Hansen Holding sustain.txt,0,1,10,0,1,0,0


# Output søgeresult

In [7]:
from datetime import datetime
name = datetime.now().strftime("%d/%m/%Y, %H:%M:%S")
filename = "Tekstsøgninger/Bæredygtighedsrapporter tekstsøgning " + name.replace("/","_").replace(", ","__").replace(":","_")
filename

'Tekstsøgninger/Bæredygtighedsrapporter tekstsøgning 25_02_2022__11_46_12'

In [8]:
df.to_csv(filename + ".csv", encoding = "utf-8")